# Processing Images

Here we enhance the images (training and testing) to improve the image quality.

The processing steps:
1. Reads the input image
2. Converts from BGR to RGB
3. Resizes to specified dimensions (224x224)
4. Applies CLAHE contrast enhancement (essential for retinal images)
5. Saves the processed image


In [ ]:
import cv2
import numpy as np
from pathlib import Path

class ODIRImageProcessor:
    """Simple processor for ODIR retinal images"""
    
    def __init__(self, output_size=512):
        self.output_size = output_size
    
    def process(self, input_path, output_path=None):
        """
        Process a single ODIR image
        
        Args:
            input_path: Path to input image
            output_path: Optional path to save result. If None, returns image array
        
        Returns:
            Processed image as numpy array
        """
        # Read image
        img = cv2.imread(str(input_path))
        if img is None:
            raise ValueError(f"Cannot read image: {input_path}")
        
        # Convert BGR to RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Resize
        img = cv2.resize(img, (self.output_size, self.output_size), 
                        interpolation=cv2.INTER_CUBIC)
        
        # Apply CLAHE for contrast enhancement
        img = self.apply_clahe(img)
        
        # Convert back to BGR for saving/display
        img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        # Save if output path is provided
        if output_path:
            cv2.imwrite(str(output_path), img_bgr)
            print(f"Saved processed image to: {output_path}")
        
        return img_bgr
    
    def apply_clahe(self, image):
        """Apply CLAHE contrast enhancement"""
        # Convert to LAB color space
        lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
        l_channel, a, b = cv2.split(lab)
        
        # Apply CLAHE to L-channel
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        l_channel = clahe.apply(l_channel)
        
        # Merge back
        lab = cv2.merge([l_channel, a, b])
        return cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)
    
    def process_folder(self, input_folder, output_folder, extension=".jpg"):
        """
        Process all images in a folder
        """
        input_folder = Path(input_folder)
        output_folder = Path(output_folder)
        output_folder.mkdir(parents=True, exist_ok=True)
        
        image_files = list(input_folder.glob(f"*{extension}"))
        print(f"Found {len(image_files)} images to process")
        
        for img_path in image_files:
            output_path = output_folder / f"processed_{img_path.name}"
            try:
                self.process(img_path, output_path)
            except Exception as e:
                print(f"Error processing {img_path.name}: {e}")

# Usage example
if __name__ == "__main__":
    # Initialize processor
    processor = ODIRImageProcessor(output_size=512)
    
    # Process single image
    processor.process(
        input_path="path/to/your/image.jpg",
        output_path="path/to/save/processed_image.jpg"
    )
    
    # OR process entire folder
    # processor.process_folder(
    #     input_folder="path/to/images",
    #     output_folder="path/to/processed_images"
    # )